![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

# Machine Learning

Check out

https://www.tandfonline.com/doi/full/10.1080/24751839.2021.1977066

https://towardsdatascience.com/guide-to-building-a-college-basketball-machine-learning-model-in-python-1c70b83acb51

https://towardsdatascience.com/nba-data-science-93e0314bb45e

---

## Exercise



---

Now you can complete your [data science project](10-data-analysis-project.ipynb).

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)